In [5]:
import numpy as np
import joblib
from PIL import Image
import cv2

from skimage.color import rgb2gray
# coding: utf-8

import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data

def fast_glcm(img, vmin=0, vmax=255, nbit=8, kernel_size=5):
    mi, ma = vmin, vmax
    ks = kernel_size
    h,w = img.shape

    # digitize
    bins = np.linspace(mi, ma+1, nbit+1)
    gl1 = np.digitize(img, bins) - 1
    gl2 = np.append(gl1[:,1:], gl1[:,-1:], axis=1)

    # make glcm
    glcm = np.zeros((nbit, nbit, h, w), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1

    kernel = np.ones((ks, ks), dtype=np.uint8)
    for i in range(nbit):
        for j in range(nbit):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)

    glcm = glcm.astype(np.float32)
    return glcm



def fast_glcm_contrast(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm contrast
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    cont = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            cont += glcm[i,j] * (i-j)**2

    return cont


def fast_glcm_dissimilarity(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm dissimilarity
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    diss = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            diss += glcm[i,j] * np.abs(i-j)

    return diss


def fast_glcm_homogeneity(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm homogeneity
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    homo = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            homo += glcm[i,j] / (1.+(i-j)**2)

    return homo


def fast_glcm_ASM(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm asm, energy
    '''
    h,w = img.shape
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    asm = np.zeros((h,w), dtype=np.float32)
    for i in range(nbit):
        for j in range(nbit):
            asm  += glcm[i,j]**2

    ene = np.sqrt(asm)
    return asm, ene

def fast_glcm_entropy(img, vmin=0, vmax=255, nbit=8, ks=5):
    '''
    calc glcm entropy
    '''
    glcm = fast_glcm(img, vmin, vmax, nbit, ks)
    pnorm = glcm / np.sum(glcm, axis=(0,1)) + 1./ks**2
    ent  = np.sum(-pnorm * np.log(pnorm), axis=(0,1))
    return ent


def calculate_glcm_features(image):
    h, w = image.shape   
    glcm_contrast = fast_glcm_contrast(image)
    glcm_dissimilarity = fast_glcm_dissimilarity(image)
    glcm_homogeneity = fast_glcm_homogeneity(image)
    glcm_asm, glcm_energy = fast_glcm_ASM(image)
    glcm_entropy = fast_glcm_entropy(image)

    features = np.concatenate([glcm_contrast.ravel(),
                               glcm_dissimilarity.ravel(),
                               glcm_homogeneity.ravel(),
                               glcm_energy.ravel(),
                               glcm_entropy.ravel()])
    return features




from skimage.color import rgb2gray
# coding: utf-8

import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data
import joblib


def predict_class_with_threshold(image_path, model_path, target_names, threshold=0.6):
    # Load the saved model
    loaded_model = joblib.load(model_path)
    
    # Read and preprocess the new image
    new_img = np.array(Image.open(image_path).convert('L'))
    image_size = (128, 128)
    resizedimage = cv2.resize(new_img, image_size)
    
    # Calculate GLCM features for the new image
    new_image_features = calculate_glcm_features(resizedimage)  # Assuming this function is defined
    
    # Use the trained model to predict the class of the new image
    predicted_class = loaded_model.predict([new_image_features])[0]
    predicted_class_name = target_names[predicted_class]
    
    # Predict probabilities for each class for the new image
    class_probabilities = loaded_model.predict_proba([new_image_features])
    
    # Get the maximum probability
    max_prob = np.max(class_probabilities)
    
    # Check if the maximum probability is above the threshold
    if max_prob >= threshold:
        return predicted_class_name, max_prob
    else:
        return None, None

# Example usage:
image_path = r"C:\Users\nitro\Desktop\320760200_477786031135927_5576665356096875983_n.jpg"
model_path = r'U:\GLCM\final_randomForest\random_forest_model_final.pkl'
target_names = ['Blight', 'Brownspot', 'blast']
predicted_class, score = predict_class_with_threshold(image_path, model_path, target_names)

if predicted_class is not None:
    print(f"Predicted Class: {predicted_class}")
    print(f"Score: {score}")
else:
    print("The image is not confidently classified.")


The image is not confidently classified.


NameError: name 'Image' is not defined